In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import onnx

In [2]:
class my_layer(nn.Module):
    def __init__(self, B):
        super(my_layer, self).__init__()
        self.B = B
    def matmul(self, A):
        return torch.matmul(A, self.B)
    def forward(self, A):
        return self.matmul(A)

M, K, N = 1, 9216, 4096
chunk_row_size, chunk_col_size = 1, 64
input = torch.randn(M, K)
weight = torch.randn(N, K)

layer = my_layer(weight.t())
res = layer(input)
# correctness
print(torch.sum(res - torch.matmul(input, weight.t())))

tensor(0.)


In [6]:
print('==================================================================================================================================================')
print('= HW 2-3-1 create a subgraph (1) that consist of a single Linear layer of size MxKxN                                                             =')
print('==================================================================================================================================================')

print('output file:', "noraml_linear.onnx")

torch.onnx.export(layer, input, "noraml_linear.onnx", verbose=True)

= HW 2-3-1 create a subgraph (1) that consist of a single Linear layer of size MxKxN                                                             =
Exported graph: graph(%A : Float(1, 9216, strides=[9216, 1], requires_grad=0, device=cpu)):
  %/Constant_output_0 : Float(9216, 4096, strides=[1, 9216], requires_grad=0, device=cpu) = onnx::Constant[value=<Tensor>, onnx_name="/Constant"](), scope: __main__.my_layer:: # /tmp/ipykernel_180649/3490729354.py:6:0
  %2 : Float(1, 4096, strides=[4096, 1], requires_grad=0, device=cpu) = onnx::MatMul[onnx_name="/MatMul"](%A, %/Constant_output_0), scope: __main__.my_layer:: # /tmp/ipykernel_180649/3490729354.py:6:0
  return (%2)

